In [1]:
from fn_cfg import *
import params as cfg
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
import joblib

In [2]:
#dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_20s.csv')
#dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_30s.csv')
dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_10s.csv')
X = dataset.drop('eye_status', 1)
y = dataset['eye_status'].array

/var/folders/ry/jhpq361x33g2tv7b7l6j0md80000gn/T/ipykernel_26005/3062680911.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = dataset.drop('eye_status', 1)


In [3]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
pca = PCA()
dfx_pca = pca.fit(X_scaled)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)
idx = np.where(explained_variance > 0.1)[0]
print(X.columns.values[idx])
X_pca = X_scaled[:,idx]

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.20)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

# Data Scaling/normalization of the features that will go to the NN
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


#----------------------- Building the model -----------------------#

# Initializing the ANN by calling the Sequential class fromm keras of Tensorflow
ann = tf.keras.models.Sequential()

#----------------------------------------------------------------------------------
# Adding "fully connected" INPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))


#----------------------------------------------------------------------------------
# Adding "fully connected" SECOND layer to the Sequential AMM by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))


#----------------------------------------------------------------------------------
# Adding "fully connected" OUTPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 1 and Activation Function = Sigmoid
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

#----------------------- Training the model -----------------------#
# Compiling the ANN
# Type of Optimizer = Adam Optimizer, Loss Function =  crossentropy for binary dependent variable, and Optimization is done w.r.t. accuracy
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN model on training set  (fit method always the same)
# batch_size = 32, the default value, number of epochs  = 100
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

#show the vector of predictions and real values
#probabilities
y_pred_prob = ann.predict(X_test)

#probabilities to binary
y_pred = (y_pred_prob > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

#Confusion Matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix", confusion_matrix)
print("Accuracy Score", accuracy_score(y_test, y_pred))

[4.25147932e-01 1.81257137e-01 1.09932258e-01 8.14137382e-02
 4.66474946e-02 3.57008967e-02 3.47149909e-02 2.36689957e-02
 1.95671661e-02 1.29694299e-02 1.09081428e-02 9.50720648e-03
 4.95976469e-03 1.99975836e-03 1.60346990e-03 1.61685116e-06
 9.99941308e-10 1.57971457e-10 3.55778445e-11 1.86504064e-12]
['std_delta']


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

# Data Scaling/normalization of the features that will go to the NN
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


#----------------------- Building the model -----------------------#

# Initializing the ANN by calling the Sequential class fromm keras of Tensorflow
ann = tf.keras.models.Sequential()

#----------------------------------------------------------------------------------
# Adding "fully connected" INPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))


#----------------------------------------------------------------------------------
# Adding "fully connected" SECOND layer to the Sequential AMM by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 6 and Activation Function = Rectifier
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))


#----------------------------------------------------------------------------------
# Adding "fully connected" OUTPUT layer to the Sequential ANN by calling Dense class
#----------------------------------------------------------------------------------
# Number of Units = 1 and Activation Function = Sigmoid
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

#----------------------- Training the model -----------------------#
# Compiling the ANN
# Type of Optimizer = Adam Optimizer, Loss Function =  crossentropy for binary dependent variable, and Optimization is done w.r.t. accuracy
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN model on training set  (fit method always the same)
# batch_size = 32, the default value, number of epochs  = 100
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

#show the vector of predictions and real values
#probabilities
y_pred_prob = ann.predict(X_test)

#probabilities to binary
y_pred = (y_pred_prob > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

#Confusion Matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix", confusion_matrix)
print("Accuracy Score", accuracy_score(y_test, y_pred))

In [ ]:
Random.best_estimator_.fit(X_train, y_train)

In [ ]:
# save the model to disk
filename = 'best_SVC_model.sav'
joblib.dump(Random.best_estimator_, filename)

# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)

In [ ]:
#y_pred = Random.best_estimator_.predict(X_train)
y_pred = loaded_model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

In [ ]:
y_pred = loaded_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

In [ ]:
def averageProcessedEEG(device_version,scan_ID,local_path,fs,line,lowcut,highcut):
    device = importFile.neurocatch()
    fileObjects = device.init(device_version,scan_ID,local_path,dispIMG=False)
    rawEEG = fileObjects[0]
    rawEOG = fileObjects[1]
    filtering = filters()
    adaptiveFilterOutput = filtering.adaptive(rawEEG,rawEOG)
    notchFilterOutput = filtering.notch(adaptiveFilterOutput,line,fs)
    bandPassFilterOutput = filtering.butterBandPass(notchFilterOutput,lowcut,highcut,fs)[:,1]
    windowEEG = slidingWindow(bandPassFilterOutput,5000,5000)
    return windowEEG


filename = "No_Threshold.csv"
device_version = 1.0
lowcut = cfg.highPass
highcut = cfg.lowPass
sfreq = cfg.fs
localPath_scans = "/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/quality scans/"
file_dir = localPath_scans+filename
quality_scans = pd.read_csv(file_dir)
base_ND = [x for x in (quality_scans['Baseline No Dementia'].to_numpy()) if str(x) != 'nan']


scans_windowEEG = []
for i in range(len(base_ND)):
    scan_ID = base_ND[i]
    windowEEG = averageProcessedEEG(device_version,scan_ID,localPath_scans,sfreq,line,lowcut,highcut)
    scans_windowEEG.append(windowEEG)

In [ ]:
def featureExtraction(data):
    filtering = filters()
    delta_rhythms = filtering.butterBandPass(data,0.5,4,fs)
    theta_rhythms = filtering.butterBandPass(data,4,8,fs)
    alpha_rhythms = filtering.butterBandPass(data,8,13,fs)
    beta_rhythms = filtering.butterBandPass(data,13,32,fs)
    gamma_rhythms = filtering.butterBandPass(data,32,79,fs)

    std_delta_rhythms = np.std(delta_rhythms,axis=0)
    std_theta_rhythms = np.std(theta_rhythms,axis=0)
    std_alpha_rhythms = np.std(alpha_rhythms,axis=0)
    std_beta_rhythms = np.std(beta_rhythms,axis=0)
    std_gamma_rhythms = np.std(gamma_rhythms,axis=0)

    rms_delta_rhythms = np.sqrt(np.mean(delta_rhythms**2))
    rms_theta_rhythms = np.sqrt(np.mean(theta_rhythms**2))
    rms_alpha_rhythms = np.sqrt(np.mean(alpha_rhythms**2))
    rms_beta_rhythms = np.sqrt(np.mean(beta_rhythms**2))
    rms_gamma_rhythms = np.sqrt(np.mean(gamma_rhythms**2))

    var_delta_rhythms = np.var(delta_rhythms,axis=0)
    var_theta_rhythms = np.var(theta_rhythms,axis=0)
    var_alpha_rhythms = np.var(alpha_rhythms,axis=0)
    var_beta_rhythms = np.var(beta_rhythms,axis=0)
    var_gamma_rhythms = np.var(gamma_rhythms,axis=0)

    kurtosis_delta_rhythms = stats.kurtosis(delta_rhythms,axis=0)
    kurtosis_theta_rhythms = stats.kurtosis(theta_rhythms,axis=0)
    kurtosis_alpha_rhythms = stats.kurtosis(alpha_rhythms,axis=0)
    kurtosis_beta_rhythms = stats.kurtosis(beta_rhythms,axis=0)
    kurtosis_gamma_rhythms = stats.kurtosis(gamma_rhythms,axis=0)
    #features = np.vstack((std_delta_rhythms,std_theta_rhythms,std_alpha_rhythms,std_beta_rhythms,std_gamma_rhythms,rms_delta_rhythms,rms_theta_rhythms,rms_alpha_rhythms,rms_beta_rhythms,rms_gamma_rhythms,var_delta_rhythms,var_theta_rhythms,var_alpha_rhythms,var_beta_rhythms,var_gamma_rhythms,kurtosis_delta_rhythms,kurtosis_theta_rhythms,kurtosis_alpha_rhythms,kurtosis_beta_rhythms,kurtosis_gamma_rhythms)).T
    features = [std_delta_rhythms,std_theta_rhythms,std_alpha_rhythms,std_beta_rhythms,std_gamma_rhythms,rms_delta_rhythms,rms_theta_rhythms,rms_alpha_rhythms,rms_beta_rhythms,rms_gamma_rhythms,var_delta_rhythms,var_theta_rhythms,var_alpha_rhythms,var_beta_rhythms,var_gamma_rhythms,kurtosis_delta_rhythms,kurtosis_theta_rhythms,kurtosis_alpha_rhythms,kurtosis_beta_rhythms,kurtosis_gamma_rhythms]
    return features


features = []
for i in range(len(allEEG_EO)):
    features.append(featureExtraction(allEEG_EO[i]))